In [2]:
from ultralytics.data.converter import convert_coco
from ultralytics import YOLO

### Data Converting ###

In [4]:
# Define paths
coco_annotation_file = "./coco_dataset/"  # Path to COCO annotation JSON file
output_folder = "./yolo_dataset_final"  # Path to save YOLO-formatted dataset

# Convert COCO annotations to YOLO format
convert_coco(labels_dir=coco_annotation_file, save_dir=output_folder, use_segments=True)


Annotations /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/coco_dataset/instances_default.json: 100%|██████████| 108/108 [00:00<00:00, 6373.87it/s]

COCO data converted successfully.
Results saved to /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset_final


### Data Processing ###

In [5]:
import random
import shutil
import os

def split_dataset(image_folder, label_folder, mask_folder, train_folder, val_folder, val_ratio=0.2):
    """
    Split the dataset into train and validation sets.

    Args:
    - image_folder (str): Folder containing the images
    - label_folder (str): Folder containing the label files
    - mask_folder (str): Folder containing the mask files
    - train_folder (str): Folder to store the training images and labels
    - val_folder (str): Folder to store the validation images and labels
    - val_ratio (float): Proportion of the data to use for validation (default is 0.2)
    """
    # Get all image files
    images = [f for f in os.listdir(image_folder) if f.endswith('.jpeg') or f.endswith('.png')]

    # Shuffle the images to randomize the split
    random.shuffle(images)

    # Split the dataset
    val_size = int(len(images) * val_ratio)
    val_images = images[:val_size]
    train_images = images[val_size:]

    # Create train and val directories
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'masks'), exist_ok=True)

    # Move the images, labels, and masks to the appropriate folder
    for image in train_images:
        shutil.copy(os.path.join(image_folder, image), os.path.join(train_folder, 'images', image))
        
        # Copy the corresponding label file
        label_file = image.replace('.jpeg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(train_folder, 'labels', label_file))
        
        # Copy the corresponding mask file
        mask_file = image.replace('.jpeg', '.png').replace('.png', '.png')
        shutil.copy(os.path.join(mask_folder, mask_file), os.path.join(train_folder, 'masks', mask_file))

    for image in val_images:
        shutil.copy(os.path.join(image_folder, image), os.path.join(val_folder, 'images', image))
        
        # Copy the corresponding label file
        label_file = image.replace('.jpeg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(label_folder, label_file), os.path.join(val_folder, 'labels', label_file))
        
        # Copy the corresponding mask file
        mask_file = image.replace('.jpeg', '.png').replace('.png', '.png')
        shutil.copy(os.path.join(mask_folder, mask_file), os.path.join(val_folder, 'masks', mask_file))

    print(f"Dataset split complete. {len(train_images)} images for training, {len(val_images)} for validation.")

# Set the paths to your folders
image_folder = './yolo_dataset_final/images'  # Folder containing images
label_folder = './yolo_dataset_final/labels'  # Folder containing labels
mask_folder = './yolo_dataset_final/masks'    # Folder containing masks
train_folder = './yolo_dataset_final/train'  # Folder to store training images/labels/masks
val_folder = './yolo_dataset_final/val'      # Folder to store validation images/labels/masks

# Split the dataset
split_dataset(image_folder, label_folder, mask_folder, train_folder, val_folder)


Dataset split complete. 87 images for training, 21 for validation.


### Model Training ###

In [7]:
from ultralytics import YOLO

# Load the YOLOv8 segmentation model (pretrained)
model = YOLO("yolov8n-seg.pt")

# Train model
model.train(
    data="data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device="cpu"
)

Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, forma

train: Scanning /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset_final/train/labels... 87 images, 87 backgrounds, 0 corrupt: 100%|██████████| 174/174 [00:00<00:00, 1761.83it/s]

train: New cache created: /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset_final/train/labels.cache



val: Scanning /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset_final/val/labels... 21 images, 21 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<00:00, 2446.98it/s]

val: New cache created: /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/yolo_dataset_final/val/labels.cache


...

                   all         42         21      0.998          1      0.995      0.936      0.998          1      0.995      0.962



50 epochs completed in 0.699 hours.
Optimizer stripped from runs/segment/train9/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train9/weights/best.pt, 6.8MB

Validating runs/segment/train9/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
YOLOv8n-seg summary (fused): 195 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.82s/it]


                   all         42         21      0.997          1      0.995      0.974      0.997          1      0.995      0.952
Speed: 0.5ms preprocess, 180.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train9


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15f5d3a10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    

In [8]:
results = model("./images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t04_overlay.jpeg")
results[0].show()


image 1/1 /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t04_overlay.jpeg: 640x544 1 invasion area, 65.0ms
Speed: 1.7ms preprocess, 65.0ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 544)


In [9]:
results = model("./images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t71_overlay.jpeg")
results[0].show()


image 1/1 /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t71_overlay.jpeg: 640x544 1 invasion area, 68.3ms
Speed: 2.6ms preprocess, 68.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 544)
